In [3]:
import os
import sys
import glob
import re
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import pydicom as dcm

import pylab

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
layout=Layout(width='90%')


# Root directory of the project
ROOT_DIR = os.path.abspath("../Mask_RCNN/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

Using TensorFlow backend.


In [4]:
home = str(Path.home())
data_dir = f"{home}/data/aneurysm/"
mask_dir = f"{home}/data/aneurysm/512Auto.ours.masks/"
stl_dir = f"{home}/data/aneurysm/stls.ours/"
dcm_dir = f"{home}/data/aneurysm/512Auto.ours/"
pngs_dir = f"{home}/data/aneurysm/512Auto.pngs/"
masks_pngs_dir = f"{home}/data/aneurysm/512Auto.masks.pngs/"

dirs = [data_dir,mask_dir,stl_dir,dcm_dir,pngs_dir,masks_pngs_dir]

In [5]:
# for d in dirs:
#      !mkdir -p {d}
!tree --filelimit=5 {data_dir}
!pwd

/home/amer/data/aneurysm/
├── 512Auto.masks.pngs.tar.gz
├── 512Auto.ours
│   ├── A012.dcm
│   ├── A013.dcm
│   ├── A015.dcm
│   └── A016.dcm
├── 512Auto.ours.masks
│   ├── A012.mask.dcm
│   ├── A013.mask.dcm
│   ├── A015.mask.dcm
│   └── A016.mask.dcm
└── stls.ours
    ├── A012.stl
    ├── A013.stl
    ├── A015.stl
    └── A016.stl

3 directories, 13 files
/home/amer/Code/ML_for_Medical_Image_Processing


In [6]:
reg_complete_filename = re.compile("[^/]*$") # Match a file name in a path
reg_filename = re.compile("^([^.]+)") # Match a file name in a path

def get_filename(paths,num):
    complete_filename = re.search(reg_complete_filename, paths[num]).group(0)
    return re.search(reg_filename, complete_filename).group(0)
    
def get_dicoms_paths(source_dir):
    return sorted(glob.glob(f"{source_dir}*.dcm"))

In [20]:
def get_pngs(dicoms_paths,dcm_num):
    ds = dcm.read_file(dicoms_paths[dcm_num]) # get the dicom file #dcm_num
    return ds.pixel_array

# Shows one image
def browse_dicom(source_dir,dicom_num):
    dicoms_paths = get_dicoms_paths(source_dir)
    pngs = get_pngs(dicoms_paths,dicom_num)
    last = len(pngs)-1
    print(dicoms_paths[dicom_num])
    def show_png(num):
        pylab.imshow(pngs[num],cmap=pylab.cm.bone)
    interact(show_png, num=widgets.IntSlider(min=0,max=last,step=1,layout=layout));

# Shows original and mask side by side
def browse_dicom_and_mask(original_dir,mask_dir,dicom_num):
    original_paths = get_dicoms_paths(original_dir)
    mask_paths = get_dicoms_paths(mask_dir)
    
    original_pngs = get_pngs(original_paths,dicom_num)
    mask_pngs = get_pngs(mask_paths,dicom_num)
    
    last = len(original_pngs)-1
    print(original_paths[dicom_num])
    print(mask_paths[dicom_num])
    def show_png(num):
        f = plt.figure()
        f.add_subplot(1,2, 1)
        pylab.imshow(original_pngs[num],cmap=pylab.cm.bone)
        f.add_subplot(1,2, 2)
        pylab.imshow(mask_pngs[num],cmap=pylab.cm.bone)
        plt.show(block=True)
    interact(show_png, num=widgets.IntSlider(min=0,max=last,step=1,layout=layout));

In [23]:
browse_dicom_and_mask(dcm_dir,mask_dir,1)

/home/amer/data/aneurysm/512Auto.ours/A013.dcm
/home/amer/data/aneurysm/512Auto.ours.masks/A013.mask.dcm


interactive(children=(IntSlider(value=0, description='num', layout=Layout(width='90%'), max=438), Output()), _…

In [22]:
#browse_dicom(dcm_dir,1) # Show me the MRI Dicom file #2
#browse_dicom(mask_dir,1) # Show me the Mask Dicom file #2

In [7]:
def save_dicom_to_pngs(source_dir,target_dir,dicom_num):
    paths = get_dicoms_paths(source_dir)
    dicom_file_name = get_filename(paths,dicom_num)
    pngs = get_pngs(paths,dicom_num)
    slice_count = len(pngs)-1
    for i in range(dicom_num,slice_count):
        pylab.imsave(f"{target_dir}{dicom_file_name}_{i}",pngs[i],cmap=pylab.cm.bone)
    return slice_count+1

In [8]:
def breakup_dicom(source_dir,target_dir): # Dicom to pngs
    dicom_paths = get_dicoms_paths(source_dir) # Get dicoms files
    file_count = len(dicom_paths) # Count dicom files
    for i in range(0,file_count):
        print(f"Breaking up {get_filename(dicom_paths,i)} into pngs...")
        slice_num = save_dicom_to_pngs(source_dir,target_dir,i)
        print(f"Finished {get_filename(dicom_paths,i)}! {slice_num} slices were saved.\n")
    print(f"PNGs are in {pngs_dir}")

In [9]:
#breakup_dicom(dcm_dir,pngs_dir)

In [1]:
#breakup_dicom(mask_dir,masks_pngs_dir)